In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import torch
import os
import pandas as pd
import cv2
import requests
from PIL import Image
import matplotlib.pyplot as plt

In [79]:
test_df = pd.read_csv('/home/vishwa/amzn-ml/student_resource 3/dataset/train.csv')

In [80]:
test_df = test_df[:100]

In [81]:
test_df

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram
...,...,...,...,...
95,https://m.media-amazon.com/images/I/71XK5d3Oh9...,416664,wattage,49.0 watt
96,https://m.media-amazon.com/images/I/61kyBEJYDe...,459516,item_weight,500 milligram
97,https://m.media-amazon.com/images/I/71uQmsTESv...,459516,item_weight,500 milligram
98,https://m.media-amazon.com/images/I/71jG8BOi4W...,241608,item_weight,16.0 gram


In [82]:
test_df['image_link'] = test_df['image_link'].str.split('/').str[-1]

In [83]:
# moon dream 2
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

DEVICE = "cuda"
DTYPE = torch.float32 if DEVICE == "cpu" else torch.float16 # CPU doesn't support float16
MD_REVISION = "2024-07-23"

tokenizer = AutoTokenizer.from_pretrained("vikhyatk/moondream2", revision=MD_REVISION)
model = AutoModelForCausalLM.from_pretrained(
    "vikhyatk/moondream2", revision=MD_REVISION, trust_remote_code=True,
    attn_implementation="flash_attention_2" if DEVICE == "cuda" else None,
    torch_dtype=DTYPE, device_map={"": DEVICE}
)

In [84]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

In [85]:
def predictor(image_link, entity_name):
    image_dir = '/home/vishwa/amzn-ml/student_resource 3/train_images/'
    image = Image.open(image_dir + image_link)
    enc_image = model.encode_image(image)
    return model.answer_question(enc_image, f"Extract the {entity_name} from the text and only a single correct unit from the list of these units {entity_unit_map[entity_name]}, if you can't extract text from the image, print nothing", tokenizer)

In [86]:
from tqdm import tqdm
import os

if __name__ == "__main__":
    DATASET_FOLDER = '../dataset/'
    
    # Apply tqdm to the apply function
    tqdm.pandas(desc="Processing")
    test_df['prediction'] = test_df.progress_apply(lambda row: predictor(row['image_link'], row['entity_name']), axis=1)
    
    output_filename = os.path.join(DATASET_FOLDER, 'test_out_1.csv')
    # test_df[['index', 'prediction']].to_csv(output_filename, index=False)

Processing: 100%|██████████| 100/100 [18:24<00:00, 11.05s/it]


In [87]:
test_df

,image_link,group_id,entity_name,entity_value,prediction
0,61I9XdN6OFL.jpg,748919,item_weight,500.0 gram,0.3
1,71gSRbyXmoL.jpg,916768,item_volume,1.0 cup,centilitre
2,61BZ4zrjZXL.jpg,459516,item_weight,0.709 gram,200 mg
3,612mrlqiI4L.jpg,459516,item_weight,0.709 gram,1
4,617Tl40LOXL.jpg,731432,item_weight,1400 milligram,1400 mg
...,...,...,...,...,...
95,71XK5d3Oh9L.jpg,416664,wattage,49.0 watt,The image shows two blue tubes with white and ...
96,61kyBEJYDeL.jpg,459516,item_weight,500 milligram,500mg
97,71uQmsTESvL.jpg,459516,item_weight,500 milligram,500mg
98,71jG8BOi4WL.jpg,241608,item_weight,16.0 gram,"16g, 301g, 302g, 302g"


In [89]:
test_df[['entity_name', 'entity_value', 'prediction']].to_csv('output.csv', index=False)